In [95]:
import pandas as pd
import sys
import numpy as np
import scipy as sc
import bs4 as soup
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tokenize import TweetTokenizer
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from nltk.corpus import names
from string import punctuation

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('names')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package names to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package names is already up-to-date!


True

In [96]:
location = "Womens Clothing E-Commerce Reviews.csv"
womensreviews = pd.read_csv(location, encoding = "latin-1")

In [97]:
womensreviews.head(25)

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
5,5,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",2,0,4,General,Dresses,Dresses
6,6,858,39,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,5,1,1,General Petite,Tops,Knits
7,7,858,39,"Shimmer, surprisingly goes with lots","I ordered this in carbon for store pick up, an...",4,1,4,General Petite,Tops,Knits
8,8,1077,24,Flattering,I love this dress. i usually get an xs but it ...,5,1,0,General,Dresses,Dresses
9,9,1077,34,Such a fun dress!,"I'm 5""5' and 125 lbs. i ordered the s petite t...",5,1,0,General,Dresses,Dresses


In [98]:
womensreviews.isnull().sum()

Unnamed: 0                    0
Clothing ID                   0
Age                           0
Title                      3810
Review Text                 845
Rating                        0
Recommended IND               0
Positive Feedback Count       0
Division Name                14
Department Name              14
Class Name                   14
dtype: int64

In [99]:
cleanedwomensreviews = womensreviews.dropna(how='any')

In [100]:
cleanedwomensreviews.isnull().sum()

Unnamed: 0                 0
Clothing ID                0
Age                        0
Title                      0
Review Text                0
Rating                     0
Recommended IND            0
Positive Feedback Count    0
Division Name              0
Department Name            0
Class Name                 0
dtype: int64

In [101]:
def reviewWomensSentiment(review):
    review = review.lower()
    
    tken_review = word_tokenize(review)
    
    for token in tken_review:
        if token in punctuation:
            tken_review.remove(token)
            
    eng_stopwords = stopwords.words('english')
            
    clean_tokens = []
    for token in tken_review:
        if token not in eng_stopwords:
            clean_tokens.append(token)
    
    clean_review = ' '.join(clean_tokens)
    
    sid = SentimentIntensityAnalyzer()
    
    sid_rev = sid.polarity_scores(clean_review)
    
    r_comp = sid_rev['compound']
    
    return r_comp

In [103]:
cleanedwomensreviews['review_sentiment'] = cleanedwomensreviews['Review Text'].apply(reviewWomensSentiment)

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [104]:
def sentimentWomensReviewCategory(sent_num):
    if sent_num >= 0.2:
        return "positive"
    if sent_num <=-0.2:
        return "negative"
    else:
        return "neutral"

In [105]:
cleanedwomensreviews['sentiment_category'] = cleanedwomensreviews['review_sentiment'].apply(sentimentWomensReviewCategory)

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [106]:
cleanedwomensreviews.dropna(how='any')

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name,review_sentiment,sentiment_category
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses,0.9062,positive
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants,0.9464,positive
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses,0.9117,positive
5,5,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",2,0,4,General,Dresses,Dresses,0.9153,positive
6,6,858,39,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,5,1,1,General Petite,Tops,Knits,0.6361,positive
7,7,858,39,"Shimmer, surprisingly goes with lots","I ordered this in carbon for store pick up, an...",4,1,4,General Petite,Tops,Knits,0.5709,positive
8,8,1077,24,Flattering,I love this dress. i usually get an xs but it ...,5,1,0,General,Dresses,Dresses,0.7579,positive
9,9,1077,34,Such a fun dress!,"I'm 5""5' and 125 lbs. i ordered the s petite t...",5,1,0,General,Dresses,Dresses,0.9643,positive
10,10,1077,53,Dress looks like it's made of cheap material,Dress runs small esp where the zipper area run...,3,0,14,General,Dresses,Dresses,0.2263,positive
12,12,1095,53,Perfect!!!,More and more i find myself reliant on the rev...,5,1,2,General Petite,Dresses,Dresses,0.5106,positive


In [107]:
cleanedwomensreviews.isnull().sum()

Unnamed: 0                 0
Clothing ID                0
Age                        0
Title                      0
Review Text                0
Rating                     0
Recommended IND            0
Positive Feedback Count    0
Division Name              0
Department Name            0
Class Name                 0
review_sentiment           0
sentiment_category         0
dtype: int64

In [108]:
cleanedwomensreviews['Review Text'].iloc[0]

'I had such high hopes for this dress and really wanted it to work for me. i initially ordered the petite small (my usual size) but i found this to be outrageously small. so small in fact that i could not zip it up! i reordered it in petite medium, which was just ok. overall, the top half was comfortable and fit nicely, but the bottom half had a very tight under layer and several somewhat cheap (net) over layers. imo, a major design flaw was the net over layer sewn directly into the zipper - it c'

In [109]:
cleanedwomensreviews['Review Text'].iloc[24]

"The colors weren't what i expected either. the dark blue is much more vibrant and i just couldn't find anything to really go with it. fabric is thick and good quality. has nice weight and movement to it. the skirt just wasn't for me, in the end."

In [111]:
cleanedwomensreviews['Review Text'].iloc[2000]

'This jacket looked adorable on. it was also quite comfortable. however, it sheds and left fuzz on every dark surface it touched. sadly, this is a deal breaker for me.'

In [112]:
cleanedwomensreviews['sentiment_category'].value_counts()

positive    18594
neutral       618
negative      450
Name: sentiment_category, dtype: int64

In [113]:
cleanedwomensreviews.count()

Unnamed: 0                 19662
Clothing ID                19662
Age                        19662
Title                      19662
Review Text                19662
Rating                     19662
Recommended IND            19662
Positive Feedback Count    19662
Division Name              19662
Department Name            19662
Class Name                 19662
review_sentiment           19662
sentiment_category         19662
dtype: int64

In [114]:
cleanedwomensreviews.to_csv('EvaluatedWomensClothingReviews.csv')